In [1]:
!pip install torchattacks
!pip install pandas
!pip install openpyxl
!pip install numpy
!pip install plot 
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torch.nn as nn
import time
import pandas as pd

In [3]:
import Sources.DefaultModel as Util
import Sources.MnistModel as MnistModel
import Sources.TestUtils as TestUtils

In [4]:
Device = Util.HardwareIdentification()
TrainDataLoaderMNIST,TestDataLoaderMNIST = MnistModel.ImportData(1000)
print(len(TrainDataLoaderMNIST.dataset), len(TestDataLoaderMNIST.dataset))

cuda
60000
10000
60000 10000


In [5]:
Model = MnistModel.ConvNeuralNet(10)
Model = Model.to(Device)
Epochs = 40
Loss = nn.CrossEntropyLoss()
LearningRate = 0.001
Optimizer = torch.optim.Adam(Model.parameters(), lr=LearningRate)

In [6]:
%%script false --no-raise-error
for Epoch in range(Epochs):
    ReturnLoss = Util.Trainloop(TrainDataLoaderMNIST, Model, Loss, Optimizer, Device)
    print('{:.4f}'.format(ReturnLoss))
ModelPath = 'Models/ModelMNIST.torch'
torch.save(Model.state_dict(), ModelPath)

In [7]:
Model.load_state_dict(torch.load('Models/ModelMNIST.torch'))

<All keys matched successfully>

In [8]:
Util.TestLoop(TestDataLoaderMNIST, Model, Device)
X, y = next(iter(TestDataLoaderMNIST))

Accuracy: 99.38%


In [10]:
ActivationList = [3, 4]
CoefList = [5]
CList = [10, 100]
ProbList = [4]
SimilarityList = [0.2]
Init = 0
lr = 0.2
Steps = 100

data = []
for S in SimilarityList:
  for P in ProbList:
    for C in CList:
      for Coef in CoefList:
        for Activation in ActivationList:
          DataResult, _ = TestUtils.AttackNorms(Model, C, Steps, lr,  Coef, Activation, P, S, Init, X, y, Device)
          data += DataResult

df = pd.DataFrame(data, columns=['L0 Norm', 'L2 Norm', 'Linf Norm', 'Activation', 'Coef', 'Prob', 'c', 'Similarity', 'Pre', 'Ppred', 'Apred', 'Spre'])
df.to_excel("Results/Parameters/Parameters.xlsx")

44 3.252345 0.89922774 3 5 10 4 0.2 7 7 2 7
0 0.0 0.0 3 5 10 4 0.2 2 2 2 2
24 2.3941817 0.71084493 3 5 10 4 0.2 1 1 3 1
63 3.4114282 0.76080173 3 5 10 4 0.2 0 0 9 0
37 2.6026664 0.70217913 3 5 10 4 0.2 4 4 9 4
27 2.7760267 0.92041194 3 5 10 4 0.2 1 1 7 1
48 2.3801684 0.5081067 3 5 10 4 0.2 4 4 8 4
25 2.5203023 0.7764852 3 5 10 4 0.2 9 9 5 9
17 1.3200027 0.6212969 3 5 10 4 0.2 5 5 8 5
28 2.543865 0.89785635 3 5 10 4 0.2 9 9 4 9
50 3.957823 0.9477813 3 5 10 4 0.2 0 0 2 0
41 3.245418 0.8631778 3 5 10 4 0.2 6 6 5 6
21 1.9126452 0.7140046 3 5 10 4 0.2 9 9 4 9
43 3.4907568 0.92081654 3 5 10 4 0.2 0 0 9 0
18 2.2032802 0.74036944 3 5 10 4 0.2 1 1 3 1
29 2.5169988 0.7697438 3 5 10 4 0.2 5 5 3 5
26 2.4443083 0.7941816 3 5 10 4 0.2 9 9 7 9
51 4.3005767 0.99397564 3 5 10 4 0.2 7 7 9 7
8 0.8667436 0.49025497 3 5 10 4 0.2 3 3 8 3
47 3.4708862 0.7747972 3 5 10 4 0.2 4 4 7 4
23 2.1541677 0.84673774 3 5 10 4 0.2 9 9 4 9
34 3.1780732 0.8819772 3 5 10 4 0.2 6 6 5 6
44 2.9932318 0.7669208 3 5 10 4 0.2 6 6

In [13]:
Activation = 4
Coef = 5
Prob = 2
Similarity = 0.2
Init=1
lr = 0.02
Steps = 100


data = []
start_time = time.time()
data = TestUtils.TestLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/MNIST/ComputationTimeResults.txt")
TestUtils.SaveData(data, "Results/MNIST/Results.xlsx")

467 17.00538 0.9971762 4 5 1000 2 0.2 7 7 8 7
451 16.585442 0.9968632 4 5 1000 2 0.2 2 2 8 2
473 17.587965 0.99607146 4 5 1000 2 0.2 1 1 8 1
470 16.912752 0.9973272 4 5 1000 2 0.2 0 0 8 0
481 16.701693 0.9957217 4 5 1000 2 0.2 4 4 8 4
473 17.14375 0.9975294 4 5 1000 2 0.2 1 1 8 1
479 16.918873 0.9972762 4 5 1000 2 0.2 4 4 2 4
487 16.830122 0.99581015 4 5 1000 2 0.2 9 9 8 9
465 16.529394 0.9955907 4 5 1000 2 0.2 5 5 2 5
456 16.591782 0.99621356 4 5 1000 2 0.2 9 9 2 9
455 16.681753 0.9952406 4 5 1000 2 0.2 0 0 8 0
457 16.721786 0.9942471 4 5 1000 2 0.2 6 6 2 6
473 16.50863 0.99603885 4 5 1000 2 0.2 9 9 8 9
483 16.895527 0.9940341 4 5 1000 2 0.2 0 0 2 0
484 17.45445 0.99440336 4 5 1000 2 0.2 1 1 2 1
480 16.76317 0.99562335 4 5 1000 2 0.2 5 5 2 5
455 16.43218 0.9932692 4 5 1000 2 0.2 9 9 2 9
461 16.522593 0.9965415 4 5 1000 2 0.2 7 7 2 7
450 16.469557 0.9946395 4 5 1000 2 0.2 3 3 2 3
470 16.562521 0.9953636 4 5 1000 2 0.2 4 4 8 4
481 16.832382 0.9984131 4 5 1000 2 0.2 9 9 8 9
440 16.550987

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=1
lr = 0.2

TestedIterations = [100, 1000]

for i in range((len(TestedIterations))):
    data = []
    start_time = time.time()
    data = TestUtils.TestLoopC(Model, TestedIterations[i], lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5)
    end_time = time.time()

    TestUtils.SaveTime(start_time, end_time, "Results/MNIST/TimeXM" + str(TestedIterations[i]) +".txt")
    TestUtils.SaveData(data, "Results/MNIST/ResultXM" + str(TestedIterations[i]) + ".xlsx")
    

In [ ]:
TestUtils.SaveImages("Results/MNIST/InitialImages/", X, y)

In [ ]:
Coef=5
Activation=4
Prob = 2
Similarity = 0.2
Init=1
lr = 0.2
Steps = 10000

TestUtils.SaveLoopC(Model, Steps, lr, Coef, Activation, Prob, Similarity, Init, X, y, Device, 1000, 0.1, 1.5, "Results/MNIST/Images/")

In [ ]:
dataJSMA = []

start_time = time.time()
Data = TestUtils.TestJSMA(1,1, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/MNIST/TimeJSMAM.txt")
TestUtils.SaveData(Data, "Results/MNIST/BestJSMAM.xlsx")

In [ ]:
TestUtils.SaveJSMA(1,1, Model, X, y, Device, "Results/MNIST/JSMAImages/")

In [ ]:
Count=0
dataCW = []

start_time = time.time()
Data, AX = TestUtils.TestCW(1000, 1, 10, Model, X, y, Device)
end_time = time.time()

TestUtils.SaveTime(start_time, end_time, "Results/MNIST/TimeCWM.txt")
TestUtils.SaveData([Data], "Results/MNIST/BestCWM.xlsx")